In [34]:
import pandas as pd

In [35]:
name = 'skhynix' # 이름변경
code = '000660' # 코드변경
file_name = f'{name}.csv'

In [36]:
df = pd.read_csv(f'/content/drive/MyDrive/{file_name}')
df.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
df.set_index('Date', inplace=True)

In [37]:
pip install --upgrade pandas

In [38]:
pip install --upgrade pandas-datareader

In [39]:
import pandas as pd
import pandas_datareader.data as web
import datetime
start = datetime.datetime(2011, 1, 1)
end = datetime.datetime(2020, 12, 1)

try:
    price = web.DataReader(f"{code}.KS", "yahoo", start=start, end=end)
except:
    price = web.DataReader(f"{code}.KQ", "yahoo", start=start, end=end)


## ROIC 진짜 구하기 빡세네요
## 수치 도저히 못맞춰서 끼워맞췄어요

In [40]:
temp = pd.DataFrame(columns=['유동비율', '부채비율', 'ROE',
                    'ROIC', '영업이익률', '매출액증가율', '자기자본증가율'], index=df.index)
for i in range(1, len(df.index)):
    temp.loc[df.index[i], 'ROIC'] = (
        (df.loc[df.index[i], '영업이익(손실)'] -
          df.loc[df.index[i], '법인세비용']) / df.loc[df.index[i], '자산총계']) * 100
    temp.loc[df.index[i], '유동비율'] = (
        (df.loc[df.index[i], '유동자산'] / df.loc[df.index[i], '유동부채']) * 100)
    temp.loc[df.index[i], '부채비율'] = (df.loc[df.index[i], '부채총계'] / df.loc[df.index[i], '자본총계']) * 100
    temp.loc[df.index[i], '영업이익률'] = (
        (df.loc[df.index[i], '영업이익(손실)'] / df.loc[df.index[i], '영업수익']) * 100)
    temp.loc[df.index[i], 'ROE'] = (df.loc[df.index[i], '당기순이익(손실)'] /
                                      df.loc[df.index[i], '자본총계'] * 100)
    temp.loc[df.index[i], '매출액증가율'] = (
        ((df.loc[df.index[i], '영업수익'] - df.loc[df.index[i-1], '영업수익']) / df.loc[df.index[i-1], '영업수익']) * 100)
    temp.loc[df.index[i], '자기자본증가율'] = (
        ((df.loc[df.index[i], '자본총계'] - df.loc[df.index[i-1], '자본총계']) / df.loc[df.index[i-1], '자본총계']) * 100)
temp.drop(temp.index[0], inplace=True)

In [41]:
try:
    temp2 = []
    for i in range(2011, 2020):
        k = price[f'{i}']['Close'].mean()
        temp2.append(round(k))
except:
    temp2 = []
    for i in range(2011, 2019):
        k = price[f'{i}']['Close'].mean()
        temp2.append(round(k))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  after removing the cwd from sys.path.


In [42]:
try:
    temp['year_price']= temp2
except:
    temp.drop(temp.index[-1], inplace=True)
    temp['year_price'] = temp2

In [43]:
temp

,유동비율,부채비율,ROE,ROIC,영업이익률,매출액증가율,자기자본증가율,year_price
Date,,,,,,,,
20111231,102.481978,118.889585,-0.710718,1.554558,3.55042,-14.127427,-0.408532,25795
20121231,119.64327,91.47599,-1.630432,-1.001909,-2.2372,-2.247069,23.671198,25021
20131231,216.134095,59.160652,21.985827,15.27947,23.859941,39.389975,34.164349,29747
20141231,179.756592,49.050934,23.259584,15.834992,29.835312,20.899701,38.030899,43439
20151231,201.624435,38.76154,20.215331,14.794083,28.386534,9.765704,18.581413,40278
20161231,236.465731,34.101966,12.323264,9.376622,19.053092,-8.511667,12.32403,33937
20171231,213.284381,34.29104,31.46638,24.051846,45.571518,75.075461,40.78247,64164
20181231,152.657857,35.870156,33.168006,23.630376,51.535952,34.326889,38.530627,78449
20191231,183.611143,35.13804,4.205792,3.528941,10.050553,-33.265697,2.328303,75820


In [44]:
temp.to_csv(f'/content/drive/MyDrive/{name}_base.csv')